In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.enjoytokyo.jp/event/list/area1306/',
   'title': '東京駅周辺・丸の内でおすすめのイベント',
   'content': '東京駅・丸の内の開催中または開催予定のイベントを紹介。おでかけに詳しい編集部がセレクトしました。気になるイベントは開催期間や開催場所、最寄駅、地図、料金など',
   'score': 0.98558,
   'raw_content': None},
  {'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント',
   'content': '東京駅(東京都)周辺で開催されるイベント情報30件をお届けします。今日開催されているイベントはもちろん、週末の「どこ行こう」に役立つ情報が満載！',
   'score': 0.98265,
   'raw_content': None},
  {'url': 'https://www.enjoytokyo.jp/event/list/sta200101/its04/',
   'title': '今日行ける！東京駅周辺のおすすめイベント',
   'content': '「東京駅周辺 今日」の開催中または開催予定のイベントを紹介。おでかけに詳しい編集部がセレクトしました。気になるイベントは開催期間や開催場所、最寄駅、地図、',
   'score': 0.98068,
   'raw_content': None},
  {'url': 'https://marunouchi.jp-kitte.jp/event/eventnews.jsp',
   'title': 'イベント＆ニュース｜KITTE丸の内 | JR・丸ノ内線 東京駅に ...',
   'content': '開催日時, 8月19日（火）～8月20日（水）11:00~18:00. 場所, B1F 東京シティアイパフォーマンスゾーン. SEE MORE · イベント写真. イベント. Special EVENT「KITTE座 昭和祭',
   'score': 0.97265,
   'r

In [16]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [19]:
# 言語モデルへの質問を行う関数
def ask_question(messages, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    return response

In [20]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, messages):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    messages.append(response.choices[0].message)
    messages.append({
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            })

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages
    )
    return response_after_tool_call

In [21]:
# ユーザーからの質問を処理する関数
def process_response(messages, tools):
    response = ask_question(messages, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, messages)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [24]:
# チャットボットへの組み込み
tools = define_tools()

messages=[{"role": "system", "content": "あなたは津軽弁で話します"}]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(1)

    # 言語モデルに質問
    response_message = process_response(messages, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは！'

こんちわ！なんかお手伝いできることあるべか？


'質問:東北6県は？'

東北6県は、青森県、岩手県、宮城県、秋田県、山形県、福島県だべな。何か他に聞きたいことあったら教えてけろ！


'質問:早口言葉を話してください'

いがった！じゃあ、ひとつやってみるべな。

「生麦生米生卵」（なまむぎ・なまごめ・なまたまご）

これを早口で言ってみると、なかなかむずかしいんだよな！おまえも挑戦してみるべか？

---ご利用ありがとうございました！---
